In [1]:
import torch
import torch.nn as nn
from tqdm import trange

from bikebench.data_loading import data_loading
from bikebench.resource_utils import models_and_scalers_path
from bikebench.prediction.prediction_utils import TorchStandardScaler, Preprocessor
from bikebench.prediction.usability_predictor import get_usability_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
X_tv, Y_tv = data_loading.load_usability_cont_train()
X_tv_tens = torch.tensor(X_tv.values, dtype=torch.float32, device=device)
Y_tv_tens = torch.tensor(Y_tv.values, dtype=torch.float32, device=device)

In [3]:
scaler = TorchStandardScaler().to(device)
scaler.fit(X_tv_tens)


scaler_path = models_and_scalers_path("usability_scaler.pt")
torch.save(scaler, scaler_path)

X_tv_tens = torch.tensor(scaler.transform(X_tv_tens), dtype=torch.float32, device=device)

/tmp/ipykernel_132736/2204159992.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tv_tens = torch.tensor(scaler.transform(X_tv_tens), dtype=torch.float32, device=device)


For this prediction task, model and training setting were tuned using optimization in a different notebook. Therefore, cross-validation is not used here. Training/model settings are hard-coded here for simplicity.


In [4]:
X_train = X_tv_tens
Y_train = Y_tv_tens

In [5]:
model = get_usability_model(dropout_on = True).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002723)

# Training loop
num_epochs = 150
batch_size = 4
best_loss = float('inf')
best_model = None

bar = trange(num_epochs, desc="Training")
for epoch in bar:
    model.train()
    permutation = torch.randperm(X_train.size(0))
    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i : i + batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()

    if loss.item() < best_loss:
        best_loss = loss.item()

    bar.set_postfix({
        'loss': f'{loss.item():.4f}',
        'best_loss': f'{best_loss:.4f}'
    })

best_model = model


Training: 100%|██████████| 150/150 [00:08<00:00, 16.77it/s, loss=0.0125, best_loss=0.0023]


In [ ]:
save_path = models_and_scalers_path("usability_model_weights.pt")
# torch.save(model.state_dict(), save_path, )

In [7]:
state  = torch.load(save_path, weights_only=True)

model = get_usability_model(dropout_on = False).to(device)
model.load_state_dict(state)

<All keys matched successfully>

In [8]:
from bikebench.prediction import evaluators

evaluators.evaluate_usability(model, Preprocessor(scaler_path=scaler_path, preprocess_fn=None, device=device), device=device, target_type='cont')

0.3917580246925354